In [1]:
String userHomeDir = System.getProperty("user.home");
String localRespoUrl = "file://" + userHomeDir + "/.m2/repository/";
String langchain4jVersion = "0.36.0-SNAPSHOT";
String dotenvJavaVersion = "3.0.2";

In [2]:
%%bash 
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/dev/ai4j/openai4j
rm -rf \{userHomeDir}/Library/Jupyter/kernels/rapaio-jupyter-kernel/mima_cache/dev/langchain4j/langchain4j-open-ai

In [ ]:
%dependency /add-repo local \{localRespoUrl} release|never snapshot|always
%dependency /list-repos
%dependency /add dev.ai4j:openai4j:0.23.0-CUSTOM
%dependency /add dev.langchain4j:langchain4j-open-ai:0.36.0-CUSTOM // Custom Audio
%dependency /resolve


In [ ]:
%dependency /add dev.langchain4j:langchain4j:\{langchain4jVersion}
%dependency /add io.github.cdimascio:dotenv-java:\{dotenvJavaVersion}
%dependency /add org.slf4j:slf4j-jdk14:2.0.9
%dependency /add ch.qos.logback:logback-classic:1.5.8
%dependency /list-dependencies
%dependency /resolve

In [5]:
import io.github.cdimascio.dotenv.Dotenv;
Dotenv dotenv = Dotenv.load();
String API_KEY = dotenv.get("OPENAI_API_KEY");
String OPENAI_API_KEY = API_KEY;

In [6]:
import dev.langchain4j.model.chat.listener.ChatModelListener;
import dev.langchain4j.model.chat.listener.ChatModelResponseContext;
import dev.langchain4j.model.chat.listener.ChatModelResponse;
import dev.langchain4j.model.chat.listener.ChatModelRequest;
import dev.langchain4j.data.message.UserMessage;

import dev.ai4j.openai4j.OpenAiClient;
import dev.ai4j.openai4j.chat.ChatCompletionRequest;
import dev.ai4j.openai4j.chat.ChatCompletionResponse;
import dev.ai4j.openai4j.chat.AssistantMessage;


import javax.sound.sampled.*;
import java.io.ByteArrayInputStream;
import java.io.FileOutputStream;
import java.io.IOException;
import java.util.Arrays;
import java.util.Base64;
import static java.util.Arrays.asList;

// Custom Code
import dev.ai4j.openai4j.chat.AudioSettings;

In [ ]:
import dev.langchain4j.model.chat.ChatLanguageModel;
import dev.langchain4j.model.openai.OpenAiChatModel;
import dev.langchain4j.data.message.UserMessage;
import dev.langchain4j.data.message.AiMessage;
import dev.langchain4j.model.output.Response;

import static dev.langchain4j.model.openai.OpenAiChatModelName.GPT_4_O_MINI;

var audioSettings = new AudioSettings("nova", "wav");
var modalities = asList("text", "audio");
OpenAiClient client = OpenAiClient.builder()
        .openAiApiKey(OPENAI_API_KEY)
        .logRequests()   // リクエストをログに出力
        .logResponses()  // レスポンスをログに出力
        .build();

UserMessage userMessage = new UserMessage("Hello!");
ChatLanguageModel model = OpenAiChatModel.builder()
    .apiKey(API_KEY)
    .modelName("gpt-4o-audio-preview")
    //.modelName("gpt-4o-mini")
    //.listeners(List.of(listener))
    .logRequests(true)
    .logResponses(true) 
    .audioSettings(audioSettings)
    .modalities(modalities)
    .build();
Response<AiMessage> response = model.generate(userMessage);
AssistantMessage.AudioData audioData = (AssistantMessage.AudioData) response.metadata().get("audio");

String data = audioData.data();
// System.out.println("data: " + data);
byte[] audioBytes = Base64.getDecoder().decode(data);          
var bais = new ByteArrayInputStream(audioBytes);
var audioInputStream = AudioSystem.getAudioInputStream(bais);
Clip clip = AudioSystem.getClip();
clip.open(audioInputStream);
clip.start();
while (clip.isRunning()) {
    Thread.sleep(100);
}
clip.close();
 
System.out.println("transcript: " + audioData.transcript());